In [1]:
import pandas as pd
import os

#### Task1: Merging 12 months of raw data into 1 single file

In [2]:
files = [file for file in os.listdir('./Raw_data')] 

all_months_data = pd.DataFrame()  # this is a empty dataframe

for file in files:
    df = pd.read_csv('./Raw_data/'+ file)
    all_months_data = pd.concat([all_months_data,df])

all_months_data.to_csv('all_data.csv',index=False)

#### Task 2: Data Cleaning
#### Read the dataset and see how many records do we have

In [3]:
df = pd.read_csv('all_data.csv')
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,EC2DE40644C6B0F4,classic_bike,2022-05-23 23:06:58,2022-05-23 23:40:19,Wabash Ave & Grand Ave,TA1307000117,Halsted St & Roscoe St,TA1309000025,41.891466,-87.626761,41.943670,-87.648950,member
1,1C31AD03897EE385,classic_bike,2022-05-11 08:53:28,2022-05-11 09:31:22,DuSable Lake Shore Dr & Monroe St,13300,Field Blvd & South Water St,15534,41.880958,-87.616743,41.886349,-87.617517,member
2,1542FBEC830415CF,classic_bike,2022-05-26 18:36:28,2022-05-26 18:58:18,Clinton St & Madison St,TA1305000032,Wood St & Milwaukee Ave,13221,41.882242,-87.641066,41.907655,-87.672552,member
3,6FF59852924528F8,classic_bike,2022-05-10 07:30:07,2022-05-10 07:38:49,Clinton St & Madison St,TA1305000032,Clark St & Randolph St,TA1305000030,41.882242,-87.641066,41.884576,-87.631890,member
4,483C52CAAE12E3AC,classic_bike,2022-05-10 17:31:56,2022-05-10 17:36:57,Clinton St & Madison St,TA1305000032,Morgan St & Lake St,TA1306000015,41.882242,-87.641066,41.885779,-87.651025,member


In [4]:
df.shape

(5860776, 13)

#### Check datatypes

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5860776 entries, 0 to 5860775
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 581.3+ MB


#### Check and drop null values

In [6]:
df.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    823167
start_station_id      823164
end_station_name      878338
end_station_id        878338
start_lat                  0
start_lng                  0
end_lat                 5036
end_lng                 5036
member_casual              0
dtype: int64

In [7]:
df = df.dropna(how='any')
df.shape

(4667299, 13)

#### ride_id should be unique, check duplicates

In [8]:
df['ride_id'].duplicated().value_counts()

False    4667299
Name: ride_id, dtype: int64

#### Trim extra space

In [9]:
trim_strings = lambda x: x.strip() if isinstance(x, str) else x
df = df.applymap(trim_strings)

In [10]:
df.sample(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
703277,84BFAE84F4F91559,electric_bike,2021-09-07 10:51:35,2021-09-07 12:00:20,South Shore Dr & 67th St,KA1503000029,South Shore Dr & 74th St,KA1503000020,41.773625,-87.567323,41.762374,-87.559269,casual
1578333,C33F3A0E0EF3B23E,classic_bike,2021-07-09 17:06:05,2021-07-09 17:09:08,Sheffield Ave & Kingsbury St,13154,Halsted St & Willow St,TA1307000166,41.910522,-87.653106,41.913865,-87.648755,member
2237278,014C4234DCD69A9C,electric_bike,2021-11-17 16:43:18,2021-11-17 17:04:57,Clark St & 9th St (AMLI),SL-009,Clark St & Randolph St,TA1305000030,41.870814,-87.631059,41.884897,-87.632105,casual
5155731,42D5F9C7ACBF232A,classic_bike,2021-08-14 10:00:33,2021-08-14 10:09:38,Larrabee St & Oak St,KA1504000116,LaSalle Dr & Huron St,KP1705001026,41.900219,-87.642985,41.894877,-87.632326,casual
2849112,17E7197567C794EE,electric_bike,2022-03-18 21:05:46,2022-03-18 21:10:01,Broadway & Barry Ave,13137,Halsted St & Wrightwood Ave,TA1309000061,41.937697,-87.644098,41.929143,-87.649077,member


#### Rename the column started_at, ended_at, member_casual

In [11]:
df = df.rename(columns = {'started_at':'starting_time', 'ended_at':'ending_time', 'member_casual':'membership_type'})

#### Datetime is now in string datatype, need to convert them into datetime format

In [12]:
df['starting_time']= pd.to_datetime(df['starting_time'])
df['ending_time']= pd.to_datetime(df['ending_time'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4667299 entries, 0 to 5860775
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   starting_time       datetime64[ns]
 3   ending_time         datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  membership_type     object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 498.5+ MB


#### Check how many distinct values rideable_type and membership have, see if there is any typo

In [14]:
df['rideable_type'].unique()

array(['classic_bike', 'docked_bike', 'electric_bike'], dtype=object)

In [15]:
df['membership_type'].unique()

array(['member', 'casual'], dtype=object)

#### The format of start_station_id and end_station_id is inconsistent, check if there is any issues

In [16]:
check_id_format = df[df['start_station_name'] == 'Peoria St & Jackson Blvd']
check_id_format.head()

,ride_id,rideable_type,starting_time,ending_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,membership_type
1579,C318A1F89ED28BE9,classic_bike,2022-05-26 17:49:45,2022-05-26 18:17:41,Peoria St & Jackson Blvd,13158,Southport Ave & Wrightwood Ave,TA1307000113,41.877642,-87.649618,41.928773,-87.663913,member
2254,0543E4E62EA3F83E,classic_bike,2022-05-19 08:09:44,2022-05-19 08:15:08,Peoria St & Jackson Blvd,13158,Canal St & Monroe St,13056,41.877642,-87.649618,41.881690,-87.639530,member
2893,2DDDB439D8F5BCEE,classic_bike,2022-05-02 05:55:48,2022-05-02 05:58:57,Peoria St & Jackson Blvd,13158,Halsted St & Polk St,TA1307000121,41.877642,-87.649618,41.871840,-87.646640,member
3532,DA8348E7865A1F32,electric_bike,2022-05-21 23:59:08,2022-05-22 00:21:45,Peoria St & Jackson Blvd,13158,Damen Ave & Wabansia Ave,20.0,41.877643,-87.649473,41.910000,-87.680000,member
4633,FA6F17BDDB8C6A4E,classic_bike,2022-05-24 17:30:51,2022-05-24 17:39:40,Peoria St & Jackson Blvd,13158,Morgan Ave & 14th Pl,TA1306000002,41.877642,-87.649618,41.862378,-87.651062,member


#### Task3: Data Transformation for analyzing
#### Extract the year & month info (see when is the high season)

In [17]:
import datetime

# strftime('%Y-%m-%d')
df['year_month'] = df['starting_time'].apply(lambda x: x.strftime('%Y-%m')) 
df.sample(5)

,ride_id,rideable_type,starting_time,ending_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,membership_type,year_month
2235020,2C44BA3B8110BDC2,classic_bike,2021-11-01 15:18:52,2021-11-01 15:22:40,Blackstone Ave & Hyde Park Blvd,13398,Kimbark Ave & 53rd St,TA1309000037,41.802562,-87.590368,41.799568,-87.594747,member,2021-11
1792536,9141C489B5AEEBCF,classic_bike,2021-07-27 18:28:09,2021-07-27 18:48:21,Michigan Ave & Oak St,13042,DuSable Lake Shore Dr & Wellington Ave,TA1307000041,41.900960,-87.623777,41.936688,-87.636829,casual,2021-07
2429935,F1447B3AE2745F41,classic_bike,2021-11-30 16:50:03,2021-11-30 16:54:46,Green St & Madison St,TA1307000120,Racine Ave & Randolph St,13155,41.881892,-87.648789,41.884069,-87.656853,member,2021-11
3136669,8B620F2D6D8D8E09,classic_bike,2022-02-06 15:54:39,2022-02-06 16:05:37,Federal St & Polk St,SL-008,Wabash Ave & 16th St,SL-012,41.872078,-87.629544,41.860384,-87.625813,casual,2022-02
432857,EC86D26A80CC7F41,electric_bike,2022-05-13 14:58:53,2022-05-13 15:03:24,Wells St & Hubbard St,TA1307000151,Larrabee St & Oak St,KA1504000116,41.890164,-87.634405,41.900219,-87.642985,casual,2022-05


#### Calculate usage time in seconds (see usage pattern between casual users and membership users )

In [18]:
df['duration_seconds'] = (df['ending_time'] - df['starting_time']).dt.total_seconds()
df.head()

,ride_id,rideable_type,starting_time,ending_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,membership_type,year_month,duration_seconds
0,EC2DE40644C6B0F4,classic_bike,2022-05-23 23:06:58,2022-05-23 23:40:19,Wabash Ave & Grand Ave,TA1307000117,Halsted St & Roscoe St,TA1309000025,41.891466,-87.626761,41.943670,-87.648950,member,2022-05,2001.0
1,1C31AD03897EE385,classic_bike,2022-05-11 08:53:28,2022-05-11 09:31:22,DuSable Lake Shore Dr & Monroe St,13300,Field Blvd & South Water St,15534,41.880958,-87.616743,41.886349,-87.617517,member,2022-05,2274.0
2,1542FBEC830415CF,classic_bike,2022-05-26 18:36:28,2022-05-26 18:58:18,Clinton St & Madison St,TA1305000032,Wood St & Milwaukee Ave,13221,41.882242,-87.641066,41.907655,-87.672552,member,2022-05,1310.0
3,6FF59852924528F8,classic_bike,2022-05-10 07:30:07,2022-05-10 07:38:49,Clinton St & Madison St,TA1305000032,Clark St & Randolph St,TA1305000030,41.882242,-87.641066,41.884576,-87.631890,member,2022-05,522.0
4,483C52CAAE12E3AC,classic_bike,2022-05-10 17:31:56,2022-05-10 17:36:57,Clinton St & Madison St,TA1305000032,Morgan St & Lake St,TA1306000015,41.882242,-87.641066,41.885779,-87.651025,member,2022-05,301.0


#### Check if there is any negative values in duration_seconds<br>Drop them if there is any

In [19]:
df[df['duration_seconds'] < 0]

,ride_id,rideable_type,starting_time,ending_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,membership_type,year_month,duration_seconds
643808,BE93718DC9182ED6,classic_bike,2021-09-29 17:04:38,2021-09-29 17:04:27,Shields Ave & 28th Pl,15443,Shields Ave & 28th Pl,15443,41.842733,-87.635491,41.842733,-87.635491,member,2021-09,-11.0
684169,6E5FD2F624AC87D3,classic_bike,2021-09-01 17:49:37,2021-09-01 17:49:31,Clybourn Ave & Division St,TA1307000115,Clybourn Ave & Division St,TA1307000115,41.904613,-87.640552,41.904613,-87.640552,member,2021-09,-6.0
704807,FA4DC99A39C36D54,classic_bike,2021-09-29 16:53:34,2021-09-29 16:53:29,Financial Pl & Ida B Wells Dr,SL-010,Financial Pl & Ida B Wells Dr,SL-010,41.875024,-87.633094,41.875024,-87.633094,member,2021-09,-5.0
717660,85BC495341AB2F18,electric_bike,2021-09-01 18:45:38,2021-09-01 18:45:24,Halsted St & Dickens Ave,13192,Halsted St & Dickens Ave,13192,41.919884,-87.648791,41.919910,-87.648777,member,2021-09,-14.0
774275,4A68473D329D45C9,classic_bike,2021-09-29 18:42:50,2021-09-29 18:36:24,Ashland Ave & Division St,13061,Ashland Ave & Division St,13061,41.903450,-87.667747,41.903450,-87.667747,member,2021-09,-386.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5416017,FB0519FF5907CACC,classic_bike,2021-08-30 12:03:08,2021-08-30 12:03:00,Halsted St & Dickens Ave,13192,Halsted St & Dickens Ave,13192,41.919936,-87.648830,41.919936,-87.648830,member,2021-08,-8.0
5416021,23D6456F1268AF00,classic_bike,2021-08-21 12:29:12,2021-08-21 12:29:07,Halsted St & Dickens Ave,13192,Halsted St & Dickens Ave,13192,41.919936,-87.648830,41.919936,-87.648830,member,2021-08,-5.0
5546377,E64AE57BDD6FE595,classic_bike,2021-08-20 16:10:14,2021-08-20 16:09:29,LaSalle St & Illinois St,13430,LaSalle St & Illinois St,13430,41.890762,-87.631697,41.890762,-87.631697,member,2021-08,-45.0
5627468,9200647C37FD38B0,classic_bike,2021-08-20 16:16:58,2021-08-20 16:14:11,Sheffield Ave & Waveland Ave,TA1307000126,Sheffield Ave & Waveland Ave,TA1307000126,41.949399,-87.654529,41.949399,-87.654529,member,2021-08,-167.0


In [20]:
# Use filtering method, instead of dropping negative value
df = df[df['duration_seconds'] > 0]

#### Extract hour info (see when is rush hour)

In [21]:
df['hour'] = df['starting_time'].dt.hour

#### Get day of week info 

In [28]:
df['day_of_week'] = df['starting_time'].apply(lambda x: x.strftime('%A'))

#### Sort values based on starting time

In [22]:
df = df.sort_values(by='starting_time')

In [31]:
df.head()

,ride_id,rideable_type,starting_time,ending_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,membership_type,year_month,duration_seconds,hour,day_of_week
0,91DD8E98C53BA95B,docked_bike,2021-06-01 00:00:38,2021-06-01 00:41:34,Lake Shore Dr & Monroe St,13300,Streeter Dr & Grand Ave,13022,41.880958,-87.616743,41.892278,-87.612043,casual,2021-06,2456.0,0,Tuesday
1,07DDAE10F8C4A9B7,electric_bike,2021-06-01 00:00:59,2021-06-01 01:09:31,Wabash Ave & 9th St,TA1309000010,Wabash Ave & 9th St,TA1309000010,41.870432,-87.625719,41.870905,-87.625706,casual,2021-06,4112.0,0,Tuesday
2,0E9702B5AFFC73D0,electric_bike,2021-06-01 00:01:00,2021-06-01 00:08:19,Perry Ave & 69th St,KA1503000047,May St & 69th St,567,41.769287,-87.628208,41.768900,-87.652936,casual,2021-06,439.0,0,Tuesday
3,032100DB251FD86C,electric_bike,2021-06-01 00:02:06,2021-06-01 01:32:12,Millennium Park,13008,Wabash Ave & Adams St,KA1503000015,41.881191,-87.624104,41.879058,-87.625622,casual,2021-06,5406.0,0,Tuesday
4,B3FF187BD7319339,docked_bike,2021-06-01 00:02:58,2021-06-01 01:12:33,Wabash Ave & 9th St,TA1309000010,Wabash Ave & 9th St,TA1309000010,41.870769,-87.625734,41.870769,-87.625734,casual,2021-06,4175.0,0,Tuesday


In [32]:
df.tail()

,ride_id,rideable_type,starting_time,ending_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,membership_type,year_month,duration_seconds,hour,day_of_week
4665662,60B78F8A1BEF98DF,classic_bike,2022-05-31 23:58:08,2022-06-01 00:07:38,Marine Dr & Ainslie St,KA1504000171,Broadway & Ridge Ave,15578,41.971600,-87.650154,41.984045,-87.660274,member,2022-05,570.0,23,Tuesday
4665663,6ED3382A3CB98D86,classic_bike,2022-05-31 23:58:38,2022-06-01 00:30:36,Wabash Ave & Wacker Pl,TA1307000131,Fairbanks Ct & Grand Ave,TA1305000003,41.886875,-87.626030,41.891847,-87.620580,member,2022-05,1918.0,23,Tuesday
4665664,6F542133C328A000,classic_bike,2022-05-31 23:59:19,2022-06-01 00:22:53,Emerald Ave & 31st St,TA1309000055,Federal St & Polk St,SL-008,41.838198,-87.645143,41.872078,-87.629544,member,2022-05,1414.0,23,Tuesday
4665665,A904966008DE7AF1,electric_bike,2022-05-31 23:59:23,2022-06-01 00:00:20,Elston Ave & Wabansia Ave,TA1309000032,Elston Ave & Wabansia Ave,TA1309000032,41.912992,-87.664191,41.912919,-87.664169,member,2022-05,57.0,23,Tuesday
4665666,761981AEEA662B35,classic_bike,2022-05-31 23:59:56,2022-06-01 00:04:04,Lincoln Ave & Fullerton Ave,TA1309000058,Sedgwick St & Webster Ave,13191,41.924161,-87.646380,41.922167,-87.638888,casual,2022-05,248.0,23,Tuesday


#### Store cleaned data to another csv file

In [33]:
df.to_csv('cyclistic_cleaned_data_20220709.csv',index = False)